In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Data for Housing/CONSOLIDATED.CSV', parse_dates=['DATE'])

# **XGBoost on House Prices delineated using fips codes and monthly time series**

In [ ]:
# Separating features and the target label/outcome

X = data.drop('hp_dlog_yoy', axis=1)
y = data['hp_dlog_yoy']

date_column = X['DATE']
fips_column = X['fips']

X['DATE'] = pd.to_datetime(X['DATE']).dt.year # Assuming 'DATE' is a date column
X['DATE'] = X['DATE'].astype(int)

# Classifying training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting training and testing data chunks into DMatrix format for XGBoost

dtrain = xgb.DMatrix(data=X_train.drop(['DATE', 'fips'], axis=1), label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(data=X_test.drop(['DATE', 'fips'], axis=1), label=y_test, enable_categorical=True)

# Specifying parameters of interest and training the model using XGBoost for out of sample forecasting of house prices in every MSA county

params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'eta': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

iters = 100
model_hp = xgb.train(params, dtrain, iters)

forecast = model_hp.predict(dtest)
score = metrics.mean_squared_error(y_test, forecast)
print(score)


# Convert forecasted house prices to dataframe and compare with test values

forecasted_prices = pd.DataFrame({
    'Date': X_test['DATE'].values,
    'FIPS': X_test['fips'].values,
    'Actual_Price': y_test.values,
    'Predicted_Price': forecast
})

print(forecasted_prices)

0.001157313153903195
       Date   FIPS  Actual_Price  Predicted_Price
0      2021  55087      0.114050         0.119533
1      2019  13067      0.046139         0.063326
2      2019  16031      0.094277         0.079384
3      2011  13059     -0.090283        -0.079877
4      2021  47117      0.111910         0.123824
...     ...    ...           ...              ...
53007  2016  22121      0.028621         0.036072
53008  2013  26081      0.119243         0.060531
53009  2017  18033      0.049419         0.055672
53010  2013  42071      0.007665         0.033434
53011  2014  48051      0.062683         0.052422

[53012 rows x 4 columns]


# **Panel OLS for house price growth**

In [ ]:
pip install linearmodels

In [ ]:
from linearmodels.panel import PanelOLS

panel_data = pd.read_csv('/content/drive/MyDrive/Data for Housing/CONSOLIDATED.CSV')

panel_data['DATE'] = pd.to_datetime(panel_data['DATE'])  # Assuming 'DATE' is a date column

panel_data = panel_data.set_index(['DATE', 'fips'])

panel_data['hp_dlog_yoy -1'] = panel_data.groupby('fips')['hp_dlog_yoy'].shift(1)

panel_data = panel_data.dropna()

ADS = 100*np.log(panel_data['ADS'])
BSB = 100*np.log(panel_data['Balance_Sheet_Balances'])
MBS = 100*np.log(panel_data['MBS'])

panel_data['ADS'] = ADS
panel_data['Balance_Sheet_Balances'] = BSB
panel_data['MBS'] = MBS
panel_data['lu_ml'] = panel_data['lu_ml']


hp_growth = panel_data['hp_dlog_yoy'] - panel_data['hp_dlog_yoy -1']
endog = hp_growth
exog = panel_data[['ADS', 'Balance_Sheet_Balances', 'MBS', 'lu_ml']]

model = PanelOLS(endog, exog, time_effects=True)
results = model.fit(cov_type='clustered', cluster_entity=True)

print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                      0   R-squared:                        0.0548
Estimator:                   PanelOLS   R-squared (Between):             -953.41
No. Observations:              263232   R-squared (Within):              -0.0076
Date:                Fri, Jun 21 2024   R-squared (Overall):             -250.71
Time:                        20:58:40   Log-likelihood                 8.939e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3785.5
Entities:                         144   P-value                           0.0000
Avg Obs:                       1828.0   Distribution:                F(4,261400)
Min Obs:                       1828.0                                           
Max Obs:                       1828.0   F-statistic (robust):             16.372
                            

# **Fitting XGBoost on the PanelOLS coefficients to Forecast House Prices Growth Change**

In [ ]:
X_significant = panel_data[['Balance_Sheet_Balances', 'ADS', 'MBS']]
#X_significant['DATE'] = X_significant.set_index(['DATE', 'fips'])
# X_significant['fips'] = X_significant.index.get_level_values('fips')
X_significant = X_significant.reset_index()
X_significant['DATE'] = pd.to_datetime(X_significant['DATE']).dt.year
#X_significant['DATE'] = X_significant['DATE'].astype(int)

X1_train, X1_test, y1_train, y1_test = train_test_split(X_significant, hp_growth, test_size = 0.2, random_state = 42)

dtrain1 = xgb.DMatrix(data=X1_train, label=y1_train)
dtest1 = xgb.DMatrix(data=X1_test, label=y1_test)

params1 = {
    'objective': 'reg:squarederror',
    'max_depth': 4,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

iterations = 5000
model_hp1 = xgb.train(params1, dtrain1, iterations)

forecast1 = model_hp1.predict(dtest1)
score1 = metrics.mean_squared_error(y1_test, forecast1)

print(forecast1)
print(score1)

[ 0.00316087  0.00375869  0.00594312 ...  0.00913937 -0.00252239
  0.00032285]
4.434151504302997e-05


In [ ]:
# Dataframe for predictions

pred_df = X1_test.copy()
print(pred_df)

y1_test = y1_test.reset_index(drop=True)

pred_df['actual'] = y1_test
pred_df['predicted'] = forecast1

pred_df['DATE'] = pd.to_datetime(pred_df['DATE'], format='%Y')
#pred_df = pred_df.drop('fips', axis=1)
#pred_df = pred_df.drop('DATE', axis=1)

pred_df.reset_index(inplace=True)  # Ensure 'entity_id' and 'time_id' are columns

# Merge predictions back to the original DataFrame
df_merged = panel_data.reset_index().merge(pred_df, on=['DATE', 'fips'], how='right')
df_merged = df_merged.sort_values(by=['fips', 'DATE'])

df_merged = df_merged.dropna()

print(df_merged)


        DATE   fips  Balance_Sheet_Balances          ADS          MBS
262236  2012  56009             1488.905185  1152.106227  1364.210315
221915  2012  48325             1488.693834  1152.779437  1363.593403
62516   2012  17081             1485.335924  1131.327869  1365.538758
63106   2013  17095             1520.991569  1095.467624  1421.893559
219923  2014  48273             1522.701886  1091.346897  1424.223083
...      ...    ...                     ...          ...          ...
83447   2017  19155             1530.905220   969.153119  1437.203242
255393  2017  55029             1530.584777   838.731227  1438.484751
142971  2021  36037             1588.282144   776.089320  1462.388863
30045   2018  12045             1522.580705   778.696700  1431.838546
45448   2018  13149             1527.540174   778.696700  1435.857353

[52647 rows x 5 columns]
            DATE   fips  Unnamed: 0  hp_dlog_yoy     lu_ml        ADS_x  \
5833  2012-01-01   1001        12.0    -0.016396 -0.030780 

In [ ]:
pip install plotly

In [ ]:
# Plotting actual and predicted house prices in each county and time period

import plotly.graph_objects as go

# Check for and remove duplicates based on 'DATE' and 'fips'
df_merged_unique = df_merged.drop_duplicates(subset=['DATE', 'fips'])

pivot_actual = df_merged_unique.pivot(index='DATE', columns='fips', values='actual')
pivot_predicted = df_merged_unique.pivot(index='DATE', columns='fips', values='predicted')

# Convert the index to numerical values (e.g., using timestamps)
#pivot_actual.index = pd.to_datetime(pivot_actual.index).astype(int) // 10**9
#pivot_predicted.index = pd.to_datetime(pivot_predicted.index).astype(int) // 10**9

# Create meshgrid for plotting
date_grid, fips_grid = np.meshgrid(pivot_actual.index, pivot_actual.columns)

surface_actual = go.Surface(
    x=date_grid,
    y=fips_grid,
    z=pivot_actual.T.values,
    colorscale='Reds',
    name='Actual Prices',
    colorbar=dict(
        title='Actual Price',
        titleside='right',
        x=1.00)
)

# Create the surface plot for predicted prices
surface_predicted = go.Surface(
    x=date_grid,
    y=fips_grid,
    z=pivot_predicted.T.values,
    colorscale='Greens',
    name='Predicted Prices',
    opacity=0.8,
    colorbar=dict(
        title='Predicted Price',
        titleside='right',
        x=1.20)
)

# Create the layout
layout = go.Layout(
    title=dict(
        text='Actual and Predicted House Prices in MSA Counties in the US',
        x=0.5,  # Center the title
        xanchor='center',
        font=dict(
            size=20,
            color='black',
            family='Arial'
        )
    ),
    scene=dict(
        xaxis=dict(title='Date'),
        yaxis=dict(title='FIPS Code'),
        zaxis=dict(title='House Price')
    ),
    showlegend=True,
    width=1200,
    height=800
)

# Combine the surfaces and layout into a figure
fig = go.Figure(data=[surface_actual, surface_predicted], layout=layout)

# Show the plot
fig.show()

# **Monte Carlo Simulation**

In [ ]:
sims = 5000
horizon = 36

params = results.params
cov_matrix = results.cov

latest_data = exog.iloc[-1].values
simulation = np.zeros((sims, horizon))

# Monte Carlo

for i in range(sims):
  for t in range(horizon):
    if t == 0:
      forecast_1step = np.dot(latest_data, params)
    else:
      latest_data[1:] = simulation[i, t-1]
      forecast_1step = np.dot(latest_data, params)   # forecasts the 1st step in the simulation


      # Noise in the data
      noise = np.random.multivariate_normal(np.zeros(len(params)), cov_matrix)
      simulation[i, t] = forecast_1step + noise[0]

In [ ]:
# Calculate the mean and confidence intervals of the simulations

mean_forecast = np.mean(simulation, axis=0)
conf_int_95 = np.percentile(simulation, [2.5, 97.5], axis=0)

print("Mean Forecast:")
print(mean_forecast)
print("95% Confidence Interval:")
print(conf_int_95)

print(simulation)

# **Comparison with XGBoost Forecasts**

In [ ]:
mean_forecast1 = np.mean(forecast1)
conf_int_95_xgb = np.percentile(forecast1, [2.5, 97.5])

print("Mean Forecast:")
print(mean_forecast1)
print("95% Confidence Interval:")
print(conf_int_95_xgb)

var_forecast = np.var(simulation)
print("Monte Carlo Forecast Variance")
print(var_forecast)

var_forecast1 = np.var(forecast1, axis=0)
print("XGBoost Forecast Variance")
print(var_forecast1)



Mean Forecast:
0.00068433257
95% Confidence Interval:
[-0.01189263  0.01136722]
Monte Carlo Forecast Variance
1.1852712805408701e-07
XGBoost Forecast Variance
2.5661631e-05
